### Bibliotheken importeren

In [2]:
import numpy as np
import pandas as pd
import requests as req
from IPython.display import display
from fuzzywuzzy import fuzz

### Open Charge Map request

In [3]:
# https://openchargemap.org/site/develop/api
api_key = "bef339a4-319a-4a46-bbbe-a5f13db5bd24"
ocm_url = f"https://api.openchargemap.io/v3/poi/?output=json&key=${api_key}&countrycode=NL&maxresults=100000"
req_ocm = req.get(ocm_url)

### Open Charge Map Data Tabel

In [4]:
df_ocm_raw = pd.json_normalize(req_ocm.json())
pd.set_option("max_columns", None) # show all cols
pd.set_option("max_rows", None) # show all cols
pd.set_option('max_colwidth', None) # show full width of showing cols
pd.set_option("expand_frame_repr", False) # print cols side by side as it's supposed to be
display(df_ocm_raw.head())

,UserComments,PercentageSimilarity,MediaItems,IsRecentlyVerified,DateLastVerified,ID,UUID,ParentChargePointID,DataProviderID,DataProvidersReference,OperatorID,OperatorsReference,UsageTypeID,UsageCost,Connections,NumberOfPoints,GeneralComments,DatePlanned,DateLastConfirmed,StatusTypeID,DateLastStatusUpdate,MetadataValues,DataQualityLevel,DateCreated,SubmissionStatusTypeID,DataProvider.WebsiteURL,DataProvider.Comments,DataProvider.DataProviderStatusType.IsProviderEnabled,DataProvider.DataProviderStatusType.ID,DataProvider.DataProviderStatusType.Title,DataProvider.IsRestrictedEdit,DataProvider.IsOpenDataLicensed,DataProvider.IsApprovedImport,DataProvider.License,DataProvider.DateLastImported,DataProvider.ID,DataProvider.Title,OperatorInfo.WebsiteURL,OperatorInfo.Comments,OperatorInfo.PhonePrimaryContact,OperatorInfo.PhoneSecondaryContact,OperatorInfo.IsPrivateIndividual,OperatorInfo.AddressInfo,OperatorInfo.BookingURL,OperatorInfo.ContactEmail,OperatorInfo.FaultReportEmail,OperatorInfo.IsRestrictedEdit,OperatorInfo.ID,OperatorInfo.Title,UsageType.IsPayAtLocation,UsageType.IsMembershipRequired,UsageType.IsAccessKeyRequired,UsageType.ID,UsageType.Title,StatusType.IsOperational,StatusType.IsUserSelectable,StatusType.ID,StatusType.Title,SubmissionStatus.IsLive,SubmissionStatus.ID,SubmissionStatus.Title,AddressInfo.ID,AddressInfo.Title,AddressInfo.AddressLine1,AddressInfo.AddressLine2,AddressInfo.Town,AddressInfo.StateOrProvince,AddressInfo.Postcode,AddressInfo.CountryID,AddressInfo.Country.ISOCode,AddressInfo.Country.ContinentCode,AddressInfo.Country.ID,AddressInfo.Country.Title,AddressInfo.Latitude,AddressInfo.Longitude,AddressInfo.ContactTelephone1,AddressInfo.ContactTelephone2,AddressInfo.ContactEmail,AddressInfo.AccessComments,AddressInfo.RelatedURL,AddressInfo.Distance,AddressInfo.DistanceUnit,OperatorInfo,UsageType
0,None,None,None,True,2021-10-09T13:59:00Z,189532,73B33652-1C0B-4637-ACF7-8FCEF205DA94,None,1,None,179.0,None,4.0,None,"[{'ID': 306581, 'ConnectionTypeID': 2, 'ConnectionType': {'FormalName': 'IEC 62196-3 Configuration AA', 'IsDiscontinued': None, 'IsObsolete': None, 'ID': 2, 'Title': 'CHAdeMO'}, 'Reference': None, 'StatusTypeID': 50, 'StatusType': {'IsOperational': True, 'IsUserSelectable': True, 'ID': 50, 'Title': 'Operational'}, 'LevelID': None, 'Level': None, 'Amps': None, 'Voltage': None, 'PowerKW': None, 'CurrentTypeID': 30, 'CurrentType': {'Description': 'Direct Current', 'ID': 30, 'Title': 'DC'}, 'Quantity': 1, 'Comments': None}, {'ID': 306582, 'ConnectionTypeID': 33, 'ConnectionType': {'FormalName': 'IEC 62196-3 Configuration FF', 'IsDiscontinued': False, 'IsObsolete': False, 'ID': 33, 'Title': 'CCS (Type 2)'}, 'Reference': None, 'StatusTypeID': 50, 'StatusType': {'IsOperational': True, 'IsUserSelectable': True, 'ID': 50, 'Title': 'Operational'}, 'LevelID': None, 'Level': None, 'Amps': None, 'Voltage': None, 'PowerKW': None, 'CurrentTypeID': None, 'CurrentType': None, 'Quantity': 1, 'Comments': None}]",2.0,None,None,None,50,2021-10-09T20:38:00Z,None,1.0,2021-10-09T13:59:00Z,200,http://openchargemap.org,None,True,1,Manual Data Entry,False,True,True,Licensed under Creative Commons Attribution-ShareAlike 4.0 International (CC BY-SA 4.0),None,1,Open Charge Map Contributors,http://www.ecotap.nl/,None,None,None,False,NaN,NaN,None,None,False,179.0,Ecotap,False,True,True,4.0,Public - Membership Required,True,True,50,Operational,True,200,Submission Published,189891,Handelskade,Handelskade,None,Zutphen,Gelderland,7202CD,159,NL,EU,159,Netherlands,52.154843,6.201008,None,None,None,None,None,None,0,NaN,NaN
1,None,None,None,True,2021-09-12T10:30:00Z,189190,BBAB6AAE-16AC-4D71-B89C-C7EDEACFCA21,None,1,None,103.0,None,4.0,"€0,37/kWh","[{'ID': 305875, 'ConnectionTypeID': 25, 'ConnectionType': {'FormalName': 'IEC 62196-2 Type 2', 'IsDiscontinued': False, 'IsObsolete': False, 'ID': 25, 'Title': 'Type 2 (Socket Only)'}, 'Reference': 'NLALLEGO011633', 'StatusTypeID': 50, 'StatusType': {'IsOperational': True, 'IsUserSelectable': T

### Correct geschreven gemeente en provincie data ophalen

In [28]:
# https://download.geoportaaloverijssel.nl/download/vector/6be88637-f10f-44a4-aa5c-fc8c0f857620
df_gp = pd.read_csv('Grenzen_van_alle_Nederlandse_gemeenten_en_provincies.csv')
df_gp = df_gp[['PROVINCIENAAM', 'GEMEENTENAAM']]
df_gp.head()

,PROVINCIENAAM,GEMEENTENAAM
0,Gelderland,Nunspeet
1,Noord-Brabant,Valkenswaard
2,Overijssel,Wierden
3,Overijssel,Hengelo (O)
4,Gelderland,Renkum


### Cleaning
#### Replace empty values with None

In [6]:
df_ocm = df_ocm_raw.copy()
# df_ocm = df_ocm.replace(np.nan, None)
df_ocm = df_ocm.replace('', np.nan)

#### Connections kolom is verbeterd
Nadat de request json was opgezet, hebben we gemerkt dat de kolom 'Connections' niet goed is genormaliseerd
naar een dataframe en nog JSON data bevat. Daarom is er gekozen om een **nested dataframe** te maken waarbij
de kolom 'Connections' een dataframe op zichzelf is.

In [7]:
df_ocm['Connections'] = df_ocm.apply(lambda row: pd.json_normalize(row['Connections']), axis=1)
display(df_ocm['Connections'].head())

0           ID  ConnectionTypeID Reference  StatusTypeID LevelID Level  Amps Voltage PowerKW  CurrentTypeID  Quantity Comments     ConnectionType.FormalName ConnectionType.IsDiscontinued ConnectionType.IsObsolete  ConnectionType.ID ConnectionType.Title  StatusType.IsOperational  StatusType.IsUserSelectable  StatusType.ID StatusType.Title CurrentType.Description  CurrentType.ID CurrentType.Title  CurrentType
0  306581                 2      None            50    None  None  None    None    None           30.0         1     None  IEC 62196-3 Configuration AA                          None                      None                  2              CHAdeMO                      True                         True             50      Operational          Direct Current            30.0                DC          NaN
1  306582                33      None            50    None  None  None    None    None            NaN         1     None  IEC 62196-3 Configuration FF                         False  

#### Gemeente en provincie kolom opschonen
Tijdens het analyseren van de data, zijn we achter gekomen dat we veel data missen in de dataset.
De dataset heeft **7783** regels aan data.

Van de **7783** regels de Provincie kolom, zijn maar **296** waarden ingevuld en de overige **7487** waarden ontbreekt.

Van de **7783** regels de Gemeente kolom, zijn maar **7766** waarden ingevuld en de overige **17** waarden ontbreekt.

Nadat we de Gemeente waarden hebben gecorrigeerd, zien we dat **428** waarden verkeerd zijn ingevuld.
De hoofdreden voor deze verkeerde waarden is dat ze stedennamen of dorpsnamen hebben ingevuld die niet
gelijkstaan aan de gemeentenaam. Ook zijn de Provincie kolom aangevuld met de correcte Gemeente waarden.

In [8]:
# display(df_ocm['AddressInfo.Town'].value_counts(dropna=False))
print(f"De Data Tabel heeft {df_ocm.shape[0]} regels.")
print(f"De Provincie kolom heeft {df_ocm['AddressInfo.StateOrProvince'].notna().sum()} ingevulde waardes.")
print(f"De Provincie kolom heeft {df_ocm['AddressInfo.StateOrProvince'].isna().sum()} ontbrekende waardes.")
print(f"De Gemeente kolom heeft {df_ocm['AddressInfo.Town'].notna().sum()} ingevulde waardes.")
print(f"De Gemeente kolom heeft {df_ocm['AddressInfo.Town'].isna().sum()} ontbrekende waardes.")

De Data Tabel heeft 7783 regels.
De Provincie kolom heeft 296 ingevulde waardes.
De Provincie kolom heeft 7487 ontbrekende waardes.
De Gemeente kolom heeft 7766 ingevulde waardes.
De Gemeente kolom heeft 17 ontbrekende waardes.


In [9]:
df_ocm['cleaned.town'] = np.nan
df_ocm['cleaned.province'] = np.nan

In [29]:
def get_cleaned_province_and_town(row):
    if row['AddressInfo.Town'] is None: return row
    if row['AddressInfo.Town'] is np.nan: return row

    highest_score = 0
    highest_score_gp = None
    for i,gp in df_gp.iterrows():
        ratio = fuzz.WRatio(gp['GEMEENTENAAM'],row['AddressInfo.Town'])
        if ratio > highest_score:
            highest_score = ratio
            highest_score_gp = gp

    if highest_score >= 95:
        row['cleaned.town'] = highest_score_gp['GEMEENTENAAM']
        row['cleaned.province'] = highest_score_gp['PROVINCIENAAM']
    return row

df_ocm = df_ocm.apply(lambda row: get_cleaned_province_and_town(row), axis=1)
display(df_ocm['cleaned.town'].value_counts(dropna=False))
display(df_ocm['cleaned.province'].value_counts(dropna=False))
display(df_ocm[['AddressInfo.Town', 'cleaned.town', 'AddressInfo.StateOrProvince', 'cleaned.province']])

Den Haag                         436
Amsterdam                        434
NaN                              428
Rotterdam                        349
Utrecht                          181
Eindhoven                        128
Haarlemmermeer                   124
's-Hertogenbosch                 119
Breda                            111
Arnhem                           101
Almere                            96
Tilburg                           94
Amersfoort                        81
Alphen aan den Rijn               81
Groningen                         78
Leiden                            75
Dordrecht                         74
Apeldoorn                         72
Westland                          64
Zwolle                            63
Nieuwegein                        63
Ede                               63
Zoetermeer                        62
Delft                             61
Haarlem                           54
Zaanstad                          50
Leeuwarden                        48
A

Zuid-Holland     2042
Noord-Holland    1300
Noord-Brabant    1058
Gelderland        807
Utrecht           734
NaN               428
Overijssel        346
Limburg           315
Friesland         211
Flevoland         177
Zeeland           161
Drenthe           110
Groningen          94
Name: cleaned.province, dtype: int64

,AddressInfo.Town,cleaned.town,AddressInfo.StateOrProvince,cleaned.province
0,Zutphen,Zutphen,Gelderland,Gelderland
1,Teylingen,Teylingen,South Holland,Zuid-Holland
2,Velden,Velsen,Limburg,Noord-Holland
3,Ede,Ede,Gelderland,Gelderland
4,Tubbergen,Tubbergen,None,Overijssel
5,Hank,NaN,North Brabant,NaN
6,Hendrik-Ido-Ambacht,Hendrik-Ido-Ambacht,None,Zuid-Holland
7,Oosterhout,Oosterhout,None,Noord-Brabant
8,Bodegraven,Bodegraven-Reeuwijk,None,Zuid-Holland
9,Nieuwegein,Nieuwegein,None,Utrecht


In [27]:
# display(df_ocm[df_ocm['AddressInfo.Town'] == 's-Gravenhage'][['AddressInfo.Town', 'cleaned.town']])
# display(df_ocm[df_ocm['AddressInfo.Town'] != df_ocm['cleaned.town']][['AddressInfo.Town', 'cleaned.town']])
#
# display(df_ocm[df_ocm['cleaned.town'].isnull()]['cleaned.town'])

,AddressInfo.Town,cleaned.town


### Open Charge Map kolommen

In [11]:
df_ocm.columns

Index(['UserComments', 'PercentageSimilarity', 'MediaItems',
       'IsRecentlyVerified', 'DateLastVerified', 'ID', 'UUID',
       'ParentChargePointID', 'DataProviderID', 'DataProvidersReference',
       'OperatorID', 'OperatorsReference', 'UsageTypeID', 'UsageCost',
       'Connections', 'NumberOfPoints', 'GeneralComments', 'DatePlanned',
       'DateLastConfirmed', 'StatusTypeID', 'DateLastStatusUpdate',
       'MetadataValues', 'DataQualityLevel', 'DateCreated',
       'SubmissionStatusTypeID', 'DataProvider.WebsiteURL',
       'DataProvider.Comments',
       'DataProvider.DataProviderStatusType.IsProviderEnabled',
       'DataProvider.DataProviderStatusType.ID',
       'DataProvider.DataProviderStatusType.Title',
       'DataProvider.IsRestrictedEdit', 'DataProvider.IsOpenDataLicensed',
       'DataProvider.IsApprovedImport', 'DataProvider.License',
       'DataProvider.DateLastImported', 'DataProvider.ID',
       'DataProvider.Title', 'OperatorInfo.WebsiteURL',
       'OperatorIn

### Aantal laadpunten per provincie

In [12]:
df_ocm.groupby('AddressInfo.StateOrProvince')['NumberOfPoints'].sum()

AddressInfo.StateOrProvince
                                         1.0
Drente                                   4.0
Drenthe                                  7.0
FRL                                      2.0
Flevolaan                                2.0
Flevoland                               25.0
Friesland                               11.0
GLD                                      2.0
Gelderland                              35.0
Gelderland                               2.0
Groningen                               15.0
Holandia Północna                        4.0
Limburg                                 34.0
MRDH                                     3.0
NH                                      47.0
Noord Brabant                            2.0
Noord Brabant                            2.0
Noord Holand                             4.0
Noord Holland                           32.0
Noord-Brabant                           90.0
Noord-Holland                           54.0
Noord-Hooland              

#### Schoongemaakte aantal laadpunten per provincie

In [13]:
# print(df_ocm[df_ocm['AddressInfo.StateOrProvince'].isna()][['AddressInfo.StateOrProvince', 'AddressInfo.Town']])
print(df_ocm[df_ocm['AddressInfo.Town'].isna()])
# Krijg de correct geschreven provincie naam op basis van FuzzyWuzzy
# def get_correct_province_name(row):
#     name = row['AddressInfo.StateOrProvince']
#     if name is None: return None
#
#     highest_score = 0
#     highest_score_name = name
#     for province in df_gp['PROVINCIENAAM'].unique():
#         ratio = fuzz.WRatio(name, province)
#         if ratio > highest_score:
#             highest_score = ratio
#             highest_score_name = province
#
#     if highest_score >= 70: return highest_score_name
#     else: return None
#
# df_ocm['cleaned.province'] = df_ocm.apply(lambda row: get_correct_province_name(row))
# print(df_ocm[['AddressInfo.StateOrProvince', 'cleaned.province']])

In [14]:
print(df_ocm['AddressInfo.StateOrProvince'].value_counts(dropna=False))
print(df_ocm['cleaned.province'].value_counts(dropna=False))

NaN                                     7487
Zeeland                                   39
Noord Holland                             28
Flevoland                                 21
Noord-Brabant                             19
Zuid-Holland                              19
Utrecht                                   18
Gelderland                                18
Noord-Holland                             17
North Brabant                             11
Stadsgewest Haaglanden                    11
Groningen                                 10
South Holland                              8
Limburg                                    8
North Holland                              7
Overijssel                                 6
NH                                         5
Drenthe                                    5
Friesland                                  5
ZH                                         4
Zuid Holland                               4
Stadsregio Amsterdam                       3
Nordhollan

### Aantal laadpunten per gemeente

In [15]:
df_ocm.groupby('AddressInfo.Town')['NumberOfPoints'].sum()

AddressInfo.Town
 Laren                               1.0
 Leeuwarden                          2.0
 Leiderdorp                          3.0
 Made                                2.0
's Hertogenbosch                    36.0
's-Heerenberg                        1.0
's-Hertogenbosch                   162.0
't Harde                             1.0
APELDOORN                            2.0
Aa en Hunze                          4.0
Aalburg                              4.0
Aalsmeer                            25.0
Aalten                               7.0
Abbekerk                             4.0
Achtkarspelen                        8.0
Akersloot                            9.0
Alblasserdam                        14.0
Albrandswaard                       31.0
Alkmaar                             64.0
Almelo                              23.0
Almere                             113.0
Almere Stad                          2.0
Almere- Stad                         2.0
Alphen aan de Rijn                   1.0

### Type laadpalen

In [16]:
types = []
for con in df_ocm['Connections']:
    if 'CurrentType.Title' in con.columns:
        for title in con['CurrentType.Title']:
            types.append(title)
    else:
        types.append(np.nan)

unique, counts = np.unique(types, return_counts=True)

df_types = pd.DataFrame()
df_types['type'] = unique
df_types['count'] = counts
df_types.head()

,type,count
0,AC (Single-Phase),11098
1,AC (Three-Phase),460
2,DC,422
3,nan,415


### Zijn tarieven transparant?

- De meeste laadpalen zijn vaag met de specifieke tarieven
- De specificeerde tarieven zijn inconsistent met de andere laadpalen

In [17]:
df_ocm['UsageCost'].value_counts()

0.00 jaarabonnement                                                  6618
0.00 afhankelijk van pas                                              343
0.00 gratis                                                           187
Free                                                                   84
€0.25/kWh; andere tarieven voor oudere auto's                          31
0.00 onbekend                                                          19
0.00 per kWh                                                           15
0.00 per laadbeurt                                                      8
0                                                                       5
0,69 eur/kWh                                                            4
0.3388                                                                  3
€0,34/kWh                                                               3
€0,25/kWh                                                               3
Unknown                               